In [1]:
import pandas as pd
import numpy as np
#import re
#import xlrd

# fileName - iterate through entire folder :)
fileName = 'Original_FiveAssetClasses.xlsx'
#fileNameOtherDevices = 'Other Device Numbers.xls'
#fileNamePolesClassHeight = 'Poles_class_height.xlsx' # Poles_class_height table

# Read xlsx file into dataframes
with pd.ExcelFile(fileName) as xlsx:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfTransformersV1 = pd.read_excel(xlsx, 'Transformers') # 280 rows
    dfSwitchesV1 = pd.read_excel(xlsx, 'Switches') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
    dfPolesV1 = pd.read_excel(xlsx, 'Poles') # 239 rows; 'Spot Number\n' col contains unique tx ids
    dfCablesV1 = pd.read_excel(xlsx, 'UGPrimaryCables')
    dfFusesV1 = pd.read_excel(xlsx, 'Fuses') # 44 items
    dfUGStructuresV1 = pd.read_excel(xlsx,'UGStructures')


Summary = {'Transformers:': dfTransformersV1.shape, 'Switches:': dfSwitchesV1.shape,'Poles:': dfPolesV1.shape, 
           'Cables:': dfCablesV1.shape, 'Fuses:':dfFusesV1.shape, 'UGStructures:':dfUGStructuresV1.shape}
dfSummary = pd.DataFrame(Summary)

# Make one copy
dfTransformersV2 = dfTransformersV1
dfSwitchesV2 = dfSwitchesV1
dfPolesV2 = dfPolesV1
dfCablesV2 = dfCablesV1
dfFusesV2 = dfFusesV1
dfUGStructuresV2 = dfUGStructuresV1

# 17 columns dropped
dropCommonColumns = ['OBJECTID','WORKORDERID','FIELDVERIFY','COMMENTS','CREATIONUSER','DATECREATED','LASTUSER',
                     'DATEMODIFIED','WORKREQUESTID','DESIGNID','WORKLOCATIONID','WMSID','WORKFLOWSTATUS',
                     'WORKFUNCTION','GISONUMBER','GISOTYPENBR','OWNERSHIP']

#*************************#
#****DEFINITIONS**********#
#*************************#
def drop_columns(dfAssetClass, dropColumns):
    dfAssetClass = dfAssetClass.drop(dropColumns, axis=1)
    return dfAssetClass

def new_columns(dfAssetClass, numAssetRows, columnID):
    dfAssetClass[columnID] = pd.DataFrame(np.empty([numAssetRows,1]).cumsum(axis=1))
    dfAssetClass.loc[:,columnID] = np.nan
    return dfAssetClass[columnID]

#*************************#
#****DEFINITIONS**********#
#*************************#

#Drop all common columns 
dfSwitchesV2 = drop_columns(dfSwitchesV2, dropCommonColumns)
dfTransformersV2 = drop_columns(dfTransformersV2, dropCommonColumns)
dfFusesV2 = drop_columns(dfFusesV2,dropCommonColumns)
dfCablesV2 = drop_columns(dfCablesV2, dropCommonColumns)
dfUGStructuresV2 = drop_columns(dfUGStructuresV2, dropCommonColumns)
dfPolesV2 = drop_columns(dfPolesV2, dropCommonColumns)

# Make one copy
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

SummaryV2 = {'Transformers:': dfTransformers.shape, 'Switches:': dfSwitches.shape,'Poles:': dfPoles.shape, 
           'Cables:': dfCables.shape, 'Fuses:':dfFuses.shape, 'UGStructures:':dfUGStructures.shape}
dfSummaryV2 = pd.DataFrame(SummaryV2)
print(dfSummary)
print(dfSummaryV2)
# print(dfSummary-dfSummaryV2) # shows 17 cols dropped

   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       39      46      31         52             57             40
   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       22      29      14         35             40             23


In [2]:
# Save future wait times while running
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

#print(dfPoles.columns)

dropTxCols = ['ANCILLARYROLE', 'ENABLED', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 'SYMBOLROTATION', 
              'GPSDATE', 'LABELTEXT', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'GROUNDREACTANCE', 'GROUNDRESISTANCE', 
              'MAGNETIZINGREACTANCE', 'MAGNETIZINGRESISTANCE', 'HIGHSIDEGROUNDREACTANCE','HIGHSIDEGROUNDRESISTANCE', 
              'HIGHSIDEPROTECTION', 'LOCATIONTYPE','COOLINGTYPE', 'FEATURE_STATUS','KVA', 'DEMAND_KVA',
              'DEMAND_DATE_MM_DD_YYYY', 'STREET_LIGHT_FACILITY', 'HIGHSIDECONFIGURATION', 'LOWSIDECONFIGURATION',
              'LOWSIDEGROUNDRESISTANCE', 'LOWSIDEVOLTAGE', 'LATITUDE', 'LONGITUDE']

# drop columns not related to Tx
dfTransformers = drop_columns(dfTransformers,dropTxCols)
# df.query('line_race != 0')
# df = df[df.line_race != 0]

# Add additional columns and fill with NaNs
#dfPoles[''] = new_columns(dfPoles, numPolesRows,'')
numTxRows = len(dfTransformers['DEVICENUMBER'])
#dfTransformers[''] = new_columns(dfTransformers,numTxRows, '')

# Don't drop so they still exist :S
# dfTransformers['UNITS'] = new_columns(dfTransformers,numTxRows, 'UNITS')
# dfTransformers['FAULTINDICATOR'] = new_columns(dfTransformers,numTxRows, 'FAULTINDICATOR')
# dfTransformers['COMPATIBLEUNITID'] = new_columns(dfTransformers,numTxRows, 'COMPATIBLEUNITID')

dfTransformers['Asset Class'] = new_columns(dfTransformers, numTxRows,'Asset Class')
dfTransformers['HI'] = new_columns(dfTransformers, numTxRows,'HI')
dfTransformers['PRID'] = new_columns(dfTransformers, numTxRows,'PRID')
dfTransformers['IN_VALLEY'] = new_columns(dfTransformers, numTxRows,'IN_VALLEY')
dfTransformers['TX_RESIDENTIAL'] = new_columns(dfTransformers, numTxRows,'TX_RESIDENTIAL')
dfTransformers['TX_COMMERCIAL'] = new_columns(dfTransformers, numTxRows,'TX_COMMERCIAL')
dfTransformers['TX_INDUSTRIAL'] = new_columns(dfTransformers, numTxRows,'TX_INDUSTRIAL')
dfTransformers['DEVICE_RESIDENTIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_RESIDENTIAL')
dfTransformers['DEVICE_COMMERCIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_COMMERCIAL')
dfTransformers['DEVICE_INDUSTRIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_INDUSTRIAL')
dfTransformers['UPSTREAM_DEVICE'] = new_columns(dfTransformers, numTxRows,'UPSTREAM_DEVICE')
dfTransformers['PCB'] = new_columns(dfTransformers, numTxRows,'PCB')

# Rename Pole columns
dfTransformers = dfTransformers.rename(columns={'DEVICENUMBER':'ID',
                                                'PHASEDESIGNATION':'Type',
                                                'INSTALLATIONDATE':'INSTALL_YEAR',
                                                'FEEDERID':'CIRCUIT',
                                                'RATEDKVA':'KVA'})
# Separate year
dfTransformers['INSTALL_YEAR'] = dfTransformers['INSTALL_YEAR'].apply(lambda x: x.year)

# UG Tx: 2/3/5/7 - 1Ph/Ntwk/Sub/Pad 3Ph [1436,27,4,507: 1642 counts]
# To avoid index vs copy error: pd.DataFrame...necessary (spent 4 hours getting rid of the warning error!)
dfUGTransformers = pd.DataFrame(dfTransformers[(dfTransformers.SUBTYPECD == 2) | 
                                  (dfTransformers.SUBTYPECD == 3) | 
                                  (dfTransformers.SUBTYPECD == 5) | 
                                  (dfTransformers.SUBTYPECD == 7) ])

# OH Tx: 1/9/10 - 1Ph/3Ph/2Ph [1125/510/7: 1347 counts]
dfOHTransformers = pd.DataFrame(dfTransformers[(dfTransformers.SUBTYPECD == 1) | 
                                  (dfTransformers.SUBTYPECD == 9) | 
                                  (dfTransformers.SUBTYPECD == 10)])

# Replace Asset class and 'SUBTYPECD' with actual tx types
dictOHTxSubclass = {'1':'Standard 1Ph','9':'Standard 3Ph','10':'Standard 2Ph'}
dictUGTxSubclass = {'2':'Padmount 1Ph','3':'Network Submersible','5':'Submersible', '7':'Padmount 3Ph'}

numOHTxRows = len(dfOHTransformers['ID'])
numUGTxRows = len(dfUGTransformers['ID'])

#dfOHTransformers['SUBTYPECD_LOOKUP'] = new_columns(dfOHTransformers, numOHTxRows,'SUBTYPECD_LOOKUP')
# pd.DataFrame(np.empty([numOHTxRows,1]).cumsum(axis=1))
#print(dfOHTransformers.head())

dfOHTransformers['SUBTYPECD'] = dfOHTransformers['SUBTYPECD'].astype(str)
dfUGTransformers['SUBTYPECD'] = dfUGTransformers['SUBTYPECD'].astype(str)

#Try using .loc[row_indexer,col_indexer] = value instead

dfOHTransformers.loc[:,'SUBTYPECD'] = dfOHTransformers['SUBTYPECD'].apply(lambda x: dictOHTxSubclass[x])
dfUGTransformers.loc[:,'SUBTYPECD'] = dfUGTransformers['SUBTYPECD'].apply(lambda x: dictUGTxSubclass[x])

# Fill in Asset and asset subclass columns
dfOHTransformers['Asset Class'] = 'Overhead Transformer'
dfUGTransformers['Asset Class'] = 'Underground Transformer'
dfOHTransformers = dfOHTransformers.rename(columns={'SUBTYPECD':'Asset Subclass'})
dfUGTransformers = dfUGTransformers.rename(columns={'SUBTYPECD':'Asset Subclass'})

# Remaining OH Tx and UG Tx specific columns
dfOHTransformers['BANKING'] = new_columns(dfOHTransformers, numOHTxRows,'BANKING')
dfUGTransformers['PEDESTAL'] = new_columns(dfUGTransformers, numUGTxRows,'PEDESTAL')
dfUGTransformers['SWITCHABLE'] = new_columns(dfUGTransformers, numUGTxRows,'SWITCHABLE')
dfUGTransformers['SWITCH_TYPE'] = new_columns(dfUGTransformers, numUGTxRows,'SWITCH_TYPE')
print(numOHTxRows, numUGTxRows)

# print(dfOHTransformers.dtypes)
# print(dfUGTransformers.dtypes)

# Tx Domain code tables
fileNameDomainCodes_Tx = 'DomainCodes_Tx.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameDomainCodes_Tx) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfUGTxDomainCodes = pd.read_excel(xls, 'UGTransformers')
    dfOHTxDomainCodes = pd.read_excel(xls, 'OHTransformers')

# Convert to string for merge purposes
dfOHTransformers['COMPATIBLEUNITID'] = dfOHTransformers['COMPATIBLEUNITID'].astype(str)
dfUGTransformers['COMPATIBLEUNITID'] = dfUGTransformers['COMPATIBLEUNITID'].astype(str)
dfOHTxDomainCodes['COMPATIBLEUNITID'] = dfOHTxDomainCodes['COMPATIBLEUNITID'].astype(str)
dfUGTxDomainCodes['COMPATIBLEUNITID'] = dfUGTxDomainCodes['COMPATIBLEUNITID'].astype(str)
#print(dfUGTxDomainCodes.head())

#df=pd.merge(df, df, how='left', left_on='ID', right_on='Loc_No')
#df.merge(df1, on='sku', how='left')
#dfOHTransformers=pd.merge(dfOHTransformers, dfOHTxDomainCodes, how='left', on='COMPATIBLEUNITID')
#dfUGTransformers=pd.merge(dfUGTransformers, dfUGTxDomainCodes, how='left', on='COMPATIBLEUNITID')
dfOHTransformers=dfOHTransformers.merge(dfOHTxDomainCodes, how='left', on='COMPATIBLEUNITID')
dfUGTransformers=dfUGTransformers.merge(dfUGTxDomainCodes, how='left', on='COMPATIBLEUNITID')
#print(dfUGTransformers.head(2))

dropTxCols2 = ['COMPATIBLEUNITID','Description']

# drop columns and drop rows that are not UG Switch
dfOHTransformers = drop_columns(dfOHTransformers,dropTxCols2)

MasterFile = pd.ExcelWriter('V2_GIS_Tx.xlsx')
dfOHTransformers.to_excel(MasterFile, 'OHTransformer')
dfUGTransformers.to_excel(MasterFile, 'UGTransformer')
MasterFile.save()

1642 1974


In [ ]:
# pole attached assets


In [97]:
# Switches
fileNameOtherDevices = 'Other Device Numbers.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameOtherDevices) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfSwitchGears = pd.read_excel(xls, 'SWITCHGEARS') # 280 rows

dropSGcols = ['Switch Gear', 'Adrs #','Location','City','Notes','To Type','Inst. Date','Mftr.','Catalog#','Serial#',
             'DOM','Comments']

dfSwitchGears = drop_columns(dfSwitchGears, dropSGcols)
#dfSwitchGears = dfSwitchGears.dropna() # drop all rows with NaN values

dictSGassetSubclass = {'PMH-3':'Air-Insulated Live Front','PMH-5':'Air-Insulated Live Front',
                       'PMH-9':'Air-Insulated Live Front','PMH-11':'Air-Insulated Live Front',
                       'PME-9':'Air-Insulated Dead Front','PME-10':'Air-Insulated Dead Front',
                       'PME-11':'Air-Insulated Live Front','VISTA-321':'SF6-Insulated Switch',
                       'VISTA-422':'SF6-Insulated Switch','VISTA-431':'SF6-Insulated Switch',
                       '422':'S&C Elec','431':'S&C Elec','321':'S&C Elec','G&W':'G&W',
                       'NET':'Carte Elec Ltd'}
# 'Type' -> 'PMH'
# 'Loc_No' -> '149-S'
dfSwitchGears['Type'] = dfSwitchGears['Type'].fillna(method='ffill')
numSGrows = len(dfSwitchGears['Loc_No'])
dfSwitchGears['Subclass'] = new_columns(dfSwitchGears, numSGrows, 'Subclass')
dfSwitchGears =dfSwitchGears.astype(str)
dfSwitchGears['Subclass'] = dfSwitchGears['Type'].apply(lambda x: dictSGassetSubclass[x])

print(dfSwitchGears.head(4))

     Type Loc_No                  Subclass
0  PME-11  149-S  Air-Insulated Live Front
1  PME-11  165-S  Air-Insulated Live Front
2  PME-11  205-S  Air-Insulated Live Front
3  PME-11  269-F  Air-Insulated Live Front


(260, 11)
dfSG Shape: (451, 3)


In [99]:
# Replace 'Asset Subclass' col with actual names
dfSwitches['ID'] = dfSwitches['ID'].astype(str)
dfSwitches=pd.merge(dfSwitches, dfSwitchGears, how='left', left_on='ID', right_on='Loc_No')
#dfSwitches['ID'] = dfSwitchGears['Loc_No'].apply(lambda x: )
#df.merge(df1, on='sku', how='left')
# print(len(pd.unique(dfSwitchGears['Loc_No'].values.ravel()))) # 111

switchesDropMoreCols = ['Asset Subclass', 'Loc_No']
dfSwitches = dfSwitches.drop(switchesDropMoreCols, axis=1)
dfSwitches = dfSwitches.rename(columns={'Subclass':'Asset Subclass'})
#Rearrange columns
dfSwitches=dfSwitches[['Asset Class', 'Asset Subclass', 'ID','CIRCUIT','INSTALL_YEAR','TIE_FEEDER','PHASING','IN_VALLEY','HI','TX_PHASE','PRID','Type']]

MasterFile = pd.ExcelWriter('V2_GIS_Poles.xlsx')
dfSwitches.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

In [75]:
#dfSwitches['ID'] = dfSwitchGears['Loc_No'].apply(lambda x: dfSwitchGears['Loc_No'] )
#df.merge(df1, on='sku', how='left')
#print(dfSwitches[40:47])
# print(len(pd.unique(dfSwitchGears['Loc_No'].values.ravel()))) # 111

MasterFile = pd.ExcelWriter('V2_GIS_Switches.xlsx')
dfSwitches.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

In [ ]:
print(dfSwitches.shape)
print('dfSG Shape:', dfSwitchGears.shape)
print(dfSwitchGears.uniquevalues())
# print(dfSwitches.head(5))
# print(dfSwitchGears.head(10))

In [33]:
# colNames = {'Transformers:': list(dfTransformers.columns), 'Switches:': list(dfSwitches.columns),
#            'Poles:': list(dfPoles.columns), 'Cables:': list(dfCables.columns), 'Fuses:':list(dfFuses.columns),
#            'UGStructures:':list(dfUGStructures.columns)}
# dfColNames = pd.Series(colNames)
# print(dfColNames['Transformers:'])

# Cables
dropCablesCols = ['ENABLED', 'INSTALLATIONDATE', 'FEEDERID', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 
                 'LENGTHSOURCE', 'MEASUREDLENGTH', 'LENGTHUOMCODE', 'WIRECOUNT', 'SUBTYPECD', 'LABELTEXT', 
                 'COMPATIBLEUNITID', 'PHASEDESIGNATION', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'ISFEEDERTRUNK', 
                 'NEUTRALUSECD', 'FEATURE_STATUS', 'CONDUCTOR_REJUVENATION', 'SHAPE_Length']



#print(dfSwitches.head(5))


# Rename OH Tx columns

# Rename UG Tx columns

# Rename Distribution Poles columns

# Rename UG Cable columns



In [66]:
#Transformer

# separate into UG and OH



In [32]:
# Fuses

dropFusesCols = ['ANCILLARYROLE', 'ENABLED', 'INSTALLATIONDATE', 'FEEDERID', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 'GPSDATE', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLEUNITID', 'PHASEDESIGNATION', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'MAXCONTINUOUSCURRENT', 'MAXINTERRUPTINGCURRENT', 'MAXOPERATINGVOLTAGE', 'PRESENTPOSITION_R', 'PRESENTPOSITION_Y', 'PRESENTPOSITION_B', 'NORMALPOSITION_R', 'NORMALPOSITION_Y', 'NORMALPOSITION_B', 'DEVICENUMBER', 'FUSELINKSIZE', 'FEATURE_STATUS', 'SYMBOLROTATION', 'INSULATOR_MATERIAL']


In [26]:
#SwitchGrouped = dfNodesCopy.groupby('SwitchRegion')
#print(SwitchGrouped.head(3))

In [41]:
# #  Ctrl + A
# # Ctrl + / to uncomment

# # ****************************
# # A. NODES sheet
# # ****************************
# # 1. Split 'Node Id' to 'NodeID_1' and 'NodeID_2'
# dfNodes['NodeID_1'], dfNodes['NodeID_2'] = zip(*dfNodes['Node Id'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Create a 'Copy' dataframe
# dfNodesCopy = pd.DataFrame(dfNodes)
# # 3. Rename all column headers to 'Nodes_' + x
# dfNodesCopy.rename(columns=lambda x: 'Nodes_'+x, inplace=True)
# #print(dfNodes_Copy.count())

# # ****************************
# # B. MASTER SPREADSHEET
# # ****************************
# # Copy dfNodesCopy into dfMaster
# dfMaster = pd.DataFrame(dfNodesCopy)
# # print(dfMaster.count())
# # Nodes_NodeID_1 and Nodes_NodeID_2 are keys

# # ****************************
# # C. Topology sheet
# # ****************************
# # 1. No renaming here,so freate a 'copy' dataframe
# dfTopologyCopy = pd.DataFrame(dfTopology)
# # 2. Rename all column headers to 'Topology_' + x
# dfTopologyCopy.rename(columns=lambda x: 'Topology_'+x, inplace=True)
# #print(dfTopologyCopy.count())

# # 3. Combine topology sheet
# # pd.merge(frame_1, frame_2, left_on = 'county_ID', right_on = 'countyid')
# # dfFinal = 
# # Topology - more match with 'Topology_Coord. Y' over 'Topology_Coord. X'
# dfMaster = pd.merge(dfMaster, dfTopologyCopy, how='outer', left_on='Nodes_NodeID_2', right_on ='Topology_Coord. Y')
# #print(dfMaster.count())


# # ****************************
# # D. Fuses sheet 
# # ****************************
# # 1. Split 'From Node' to 'FromNode_xCoord' and 'FromNode_yCoord'
# dfFuses['FromNode_xCoord'], dfFuses['FromNode_yCoord'] = zip(*dfFuses['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Split 'To Node' to 'ToNode_FuseID' and 'ToNode_FdrID'
# dfFuses['ToNode_FuseID'], dfFuses['ToNode_FdrID'] = zip(*dfFuses['To Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 3. Create a 'Copy' dataframe
# dfFusesCopy = pd.DataFrame(dfFuses)
# # 4. Rename all column headers to 'Fuses_' + x
# dfFusesCopy.rename(columns=lambda x:'Fuses_'+x, inplace=True)
# # 5. Combine Fuses sheet with Master
# dfMaster = pd.merge(dfMaster, dfFusesCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Fuses_FromNode_xCoord')
# #print(dfMaster.count())

# # ****************************
# # D1. Output excel file - For VERIFICATION purposes
# # ****************************
# # Verify the excel file 
# # http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.to_excel.html
# # http://stackoverflow.com/questions/29974672/writing-pandas-dataframe-to-excel-with-different-formats-for-different-columns
# # MasterFile = pd.ExcelWriter('master.xlsx')
# # dfMaster.to_excel(MasterFile, 'Sheet1')
# # MasterFile.save()


In [37]:
# Ctrl + / to uncomment

# # ****************************
# # E. Switch sheet 
# # ****************************
# # 1. Split 'From Node' to 'FromNode_1' and 'FromNode_2'
# dfSwitches['FromNode_1'], dfSwitches['FromNode_2'] = zip(*dfSwitches['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Create a 'Copy' dataframe
# dfSwitchesCopy = pd.DataFrame(dfSwitches)
# # 3. Rename all column headers to 'Switches_' + x
# dfSwitchesCopy.rename(columns=lambda x:'Switches_'+x, inplace=True)
# # 4. Combine Switches sheet with Master: 
# # 4.1 First with 'Switches_FromNode_1' - NodeID_1 also has '109-D'/'7-S' switch id :)
# dfMaster = pd.merge(dfMaster, dfSwitchesCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Switches_FromNode_1')
# # 4.2 Second with 'Section Id' of FusesCopy - maybe not necessary

# # ****************************
# # F. Transformer aka "Loads" in CYME
# # ****************************
# # 
# # 1. Split 'From Node' to 'FromNode_1' and 'FromNode_2'
# dfLoads['FromNode_1'], dfLoads['FromNode_2'] = zip(*dfLoads['From Node'].apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))
# # 2. Create a 'Copy' dataframe
# dfLoadsCopy = pd.DataFrame(dfLoads)
# # 3. Rename all column headers to Loads_' + x
# dfLoadsCopy.rename(columns=lambda x:'Loads_'+x, inplace=True)
# # 4. Combine all Loads with 'FromNode_1'  with dfMaster
# dfMaster = pd.merge(dfMaster, dfLoadsCopy, how='outer', left_on='Nodes_NodeID_1', right_on ='Loads_FromNode_1')
# # 4.2 May need to combine dfLoadsCopy with dfSpotLoads if tx nameplate rating not same


# #Plot
# %matplotlib inline
# import matplotlib.pyplot as plt
# pd.options.display.mpl_style = 'default'
# #dfSwitches.boxplot()
# #dfFusesCopy.boxplot(column="Fuses_Rating(A)")
# #mydf['CigarNum'] = mydf['CigarNum'].convert_objects(convert_numeric=True)
# dfFusesCopy['Fuses_FromNode_xCoord'] = dfFusesCopy['Fuses_FromNode_xCord'].convert_objects(convert_numeric=True)
# dfFusesCopy['Fuses_FromNode_yCoord'] = dfFusesCopy['Fuses_FromNode_yCord'].convert_objects(convert_numeric=True)
# #dfFusesCopy.plot(kind='scatter', x='Fuses_FromNode_xCoord', y='Fuses_FromNode_yCoord')


# # ****************************
# # D1. Output excel file
# # ****************************
# # Verify the excel file 
# # http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.to_excel.html
# # http://stackoverflow.com/questions/29974672/writing-pandas-dataframe-to-excel-with-different-formats-for-different-columns
# #MasterFile = pd.ExcelWriter('master.xlsx')
# #dfMaster.to_excel(MasterFile, 'Sheet1')
# #MasterFile.save()


In [8]:
# ****************************
# G. PRID to each region
# ****************************
# combine PRID to Tx?

# ****************************
# H. Cable 
# ****************************
# combine cable and conductors

# ****************************
# I. Conductors  
# ****************************
# combine poles

# ****************************
# J. Poles 
# ****************************
# Output excel file

